In [59]:
import pandas as pd
import json
import numpy as np
import pandas as pd
import sklearn
import random
import pickle
import json
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD,dump,KNNWithZScore,KNNBasic
from surprise.model_selection import cross_validate

# np.random.seed=42
# np.random=42

In [60]:
with open('data.json','r')as file:
         data_file = json.load(file)

users = pd.DataFrame.from_dict(data_file[0]).astype({'user_id':int})
events = pd.DataFrame.from_dict(data_file[1]).astype({'Event_id':int})
interactions = pd.DataFrame.from_dict(data_file[2])

In [61]:
users

,user_id,preference_categories
0,2,"[Film, Art, Food]"
1,1,"[Musical, Photography, Science]"
2,3,"[Health, Outdoor, Sports]"
3,4,"[Health, Technology, Theatre]"
4,5,"[Technology, Comedy, Gaming]"
5,6,"[Theatre, Business, Lifestyle]"
6,7,"[Travel, Food, Photography]"
7,8,"[Outdoor, Technology, Travel]"
8,9,"[Health, Dance, Social]"
9,10,"[Business, Social, Travel]"


In [62]:
events

,Event_id,categories
0,71,"[Health, Literature, Musical]"
1,72,"[Health, Literature, Outdoor]"
2,73,"[Film, Outdoor, Photography]"
3,74,"[Comedy, Crafts, Other, Sports, Theatre]"
4,75,"[Comedy, Musical, Outdoor]"
5,76,"[Educational, Musical]"
6,77,"[Comedy, Outdoor]"
7,78,[Crafts]
8,79,"[Comedy, Crafts, Food, Religious, Technology]"
9,80,"[Business, Comedy, Health, Photography]"


In [63]:
interactions

,user_id,event_id,liked,interest,rate
0,2,71,0,1,NaN
1,1,72,1,0,NaN
2,1,85,1,0,NaN
3,3,71,1,1,NaN
4,3,80,1,0,NaN
5,3,82,1,0,NaN
6,3,83,1,0,NaN
7,5,74,0,0,3.0
8,5,78,0,1,NaN
9,5,80,1,0,NaN


In [64]:
interactions['interaction']=interactions['liked']|interactions['interest']

interactions.drop(['liked','interest'],axis=1,inplace=True)

rating=interactions[interactions['rate'].notnull()].drop('interaction',axis=1).astype(float)

interaction=interactions[interactions['interaction']!=0].drop('rate',axis=1).astype(int)

In [65]:
events['categories'] = events.apply(lambda x:' '.join(x['categories']),axis=1).str.lower()
users['preference_categories']=users.apply(lambda x:' '.join(x['preference_categories']),axis=1).str.lower()

In [66]:
# categories=set()
# events['categories'].apply(lambda x: categories.update(x.split()))
# categories.remove('other')
# categories=dict(zip(categories,list(range(len(categories)))))

# content Based

In [67]:
events_count_matrix = CountVectorizer(min_df=0.0,analyzer='word',ngram_range=(1,5),stop_words='english')
events_count=events_count_matrix.fit_transform(events['categories'])

users_count_matrix = CountVectorizer(vocabulary=events_count_matrix.get_feature_names_out())
users_count=users_count_matrix.fit_transform(users['preference_categories'])

In [68]:
events_count_matrix.get_feature_names_out()

array(['art', 'art fashion', 'art fashion musical',
       'art fashion musical photography',
       'art fashion musical photography theatre', 'business',
       'business comedy', 'business comedy health',
       'business comedy health photography', 'business food',
       'business food gaming', 'business food gaming travel', 'charity',
       'charity film', 'charity film sports',
       'charity film sports travel', 'comedy', 'comedy crafts',
       'comedy crafts food', 'comedy crafts food religious',
       'comedy crafts food religious technology', 'comedy crafts sports',
       'comedy crafts sports theatre', 'comedy health',
       'comedy health photography', 'comedy musical',
       'comedy musical outdoor', 'comedy outdoor', 'crafts',
       'crafts food', 'crafts food religious',
       'crafts food religious technology', 'crafts sports',
       'crafts sports theatre', 'educational', 'educational musical',
       'fashion', 'fashion musical', 'fashion musical photograph

In [69]:
#cosine Similarities 

event_cs = cosine_similarity(events_count)
user_cs=cosine_similarity(users_count,events_count)

In [70]:
user_indices = pd.Series(users.index, index=users['user_id'])
event_indices = pd.Series(events.index, index=events['Event_id'])

There is Two options 

first and by default input is user id and output recommended events based on common categories between users and events

Second option : input is event id and output recommended events based on common categories between events 

In [71]:
def get_recommendations(id,input='user',k=10):    
         try:    
                  if input == 'user':
                           idx = user_indices[id]
                           sim_scores = list(enumerate(user_cs[idx]))
                  elif input == 'event':
                           idx = event_indices[id]
                           sim_scores = list(enumerate(event_cs[idx]))
                           sim_scores.pop(0)
                  
                  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

                  sim_scores = sim_scores[:k]

                  rec_event_indices = [i[0] for i in sim_scores]
                  return events.iloc[rec_event_indices]
         
         except:
                 print('Input id is invalid for User/Event, make sure id relative to content')


In [72]:
get_recommendations(71)

Input id is invalid for User/Event, make sure id relative to content


In [73]:
users[users['user_id']==2]

,user_id,preference_categories
0,2,film art food


In [74]:
get_recommendations(id=2,input='user')

,Event_id,categories
15,69,film
2,73,film outdoor photography
10,81,charity film sports travel
12,83,business food gaming travel
8,79,comedy crafts food religious technology
11,82,art fashion musical photography theatre
0,71,health literature musical
1,72,health literature outdoor
3,74,comedy crafts other sports theatre
4,75,comedy musical outdoor


In [75]:
events[events['Event_id']==71]

,Event_id,categories
0,71,health literature musical


In [76]:
get_recommendations(id=71,input='event')

,Event_id,categories
1,72,health literature outdoor
5,76,educational musical
4,75,comedy musical outdoor
9,80,business comedy health photography
11,82,art fashion musical photography theatre
2,73,film outdoor photography
3,74,comedy crafts other sports theatre
6,77,comedy outdoor
7,78,crafts
8,79,comedy crafts food religious technology


# Collaborative

## Collborative Filtering Based on interaction

In [77]:
# create a custom class to set default prediction equal zero (user doesn't interact with any events)
class KNNBasic(KNNBasic):
        def __init__(self, k=40, min_k=1, sim_options=..., verbose=True, **kwargs):
                super().__init__(k, min_k, sim_options, verbose, **kwargs)
        
        def predict(self, uid, iid, r_ui=None, clip=True, verbose=False):
                return super().predict(uid, iid, r_ui, clip, verbose)
        
        def default_prediction(self):
                return 0

In [78]:
interaction.head(3)

,user_id,event_id,interaction
0,2,71,1
1,1,72,1
2,1,85,1


In [79]:
reader = Reader()

data = Dataset.load_from_df(interaction, reader)

sim_options = {"name": "cosine","user_based": True}

knn = KNNBasic(sim_options=sim_options,verbose=False,random_state=42)


cross_validate(knn, data, measures=['RMSE', 'MAE'],cv=5)

trainset = data.build_full_trainset()

trainset.rating_scale=(0,1)

knn.fit(trainset)
""

''

Hybried Recommender firstly get recommended events from content based model then resort according to user behavior 

In [80]:
def get_hybried_recommendations(user_id):
    recommended_events=[]

    def prediction(event_id):
        pred=knn.predict(user_id,event_id)
        if pred.est!=0:
                return pred.est, pred.details['actual_k']
        else:
                return 0,0
    
    recommended_events=get_recommendations(user_id,k=15).copy()

    recommended_events['est'] = recommended_events['Event_id'].apply(prediction)
    
    recommended_events = recommended_events.sort_values('est', ascending=False)
    
    return recommended_events

In [81]:
users[users['user_id']==3]

,user_id,preference_categories
2,3,health outdoor sports


In [82]:
# get_recommendations(3)

est variable --> (user_may_be_likes, k_similar neighbors)

In [83]:
get_hybried_recommendations(3)

,Event_id,categories,est
0,71,health literature musical,"(1, 3)"
9,80,business comedy health photography,"(1, 2)"
12,83,business food gaming travel,"(1, 2)"
7,78,crafts,"(1, 1)"
11,82,art fashion musical photography theatre,"(1, 1)"
13,84,religious,"(1, 1)"
1,72,health literature outdoor,"(0, 0)"
6,77,comedy outdoor,"(0, 0)"
2,73,film outdoor photography,"(0, 0)"
4,75,comedy musical outdoor,"(0, 0)"


### Users Similarity

In [84]:
user_similarity=knn.compute_similarities()
# pd.DataFrame(user_similarity)

In [85]:
# neareast_users get similar k users and its sorted and if its no more then fill with any users
#gets uid
def get_closest_users(uid,k=10):
         try:
               return knn.get_neighbors(uid,k)
         except:
               print('User Not Found')  

get_closest_users(0)

[2, 4, 1, 3]

________________________________________________________________

## Collborative Filtering Based on Rating

In [86]:
# reader = Reader()

# data = Dataset.load_from_df(rating, reader)

# svd = SVD()

# cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=5)

# trainset = data.build_full_trainset()

# svd.fit(trainset)

# file_name = os.path.expanduser("./collaborative_model.pkl")
# dump.dump(file_name, algo=svd)